<img src="https://i.imgur.com/6U6q5jQ.png"/>

_____
<a id='home'></a>

<a target="_blank" href="https://colab.research.google.com/github/SocialAnalytics-StrategicIntelligence/introOptimization/blob/main/Intro_To_Optimization.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>


# Introduction to Optimization for Decision Making


In [1]:
%%html
<iframe src="https://docs.google.com/presentation/d/e/2PACX-1vQHq0p2eTmxRWJjDmo1mUmdarYgIrEew4ieiVbIGQy-D_CyBw5rbbRUlRxwLKKaVQpRV9Hs8MGnz0X2/embed?start=false&loop=false&delayms=3000" frameborder="0" width="960" height="569" allowfullscreen="true" mozallowfullscreen="true" webkitallowfullscreen="true"></iframe>

# Part 1: Maximization/Minimization

Please, go to your _environment_ in Anacoda Navigator to install **glpk** and **pulp**  before runing the codes below.
Then, call the library:

In [2]:
!pip show glpk pulp
!pip install glpk pulp

Name: PuLP


Version: 2.9.0
Summary: PuLP is an LP modeler written in python. PuLP can generate MPS or LP files and call GLPK, COIN CLP/CBC, CPLEX, and GUROBI to solve linear problems.
Home-page: https://github.com/coin-or/pulp
Author: J.S. Roy and S.A. Mitchell and F. Peschiera
Author-email: pulp@stuartmitchell.com
License: 
Location: C:\Users\maria\AppData\Local\Programs\Python\Python312\Lib\site-packages
Requires: 
Required-by: spopt

  error: subprocess-exited-with-error
  
  × Building wheel for glpk (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [5 lines of output]
      running bdist_wheel
      running build
      running build_ext
      building 'glpk' extension
      error: Microsoft Visual C++ 14.0 or greater is required. Get it with "Microsoft C++ Build Tools": https://visualstudio.microsoft.com/visual-cpp-build-tools/
      [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for glpk

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: ERROR: Failed to build installable wheels for some pyproject.toml based projects (glpk)



  Using cached glpk-0.4.8.tar.gz (160 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
Failed to build glpk


In [3]:
import pulp as pp

1. **Initialize the MODEL**: just write the name and declare if it is maximization or minimization problem type.

In [4]:
model = pp.LpProblem(name='refinery-problem', # just the name
                     sense=pp.LpMaximize) # type of problem

2. **Declare the VARIABLES**: The refinery model consists of these _variables_:

In [5]:
# how much gas?
Gas = pp.LpVariable(name="Gas",  # just the name
                    lowBound=0,  # ensure non-negativity
                    cat='Continuous') # here: you accept decimal values

# how much oil?
Oil = pp.LpVariable(name="Oil",
                 lowBound=0,
                 cat='Continuous')

3. **Create function to OPTIMIZE**: The function is just the linear combination of the variables and their _given coefficients__:

In [6]:
GasCoeff=1.9
OilCoeff=1.5
obj_func = GasCoeff*Gas + OilCoeff*Oil

4. **Represent the constraints**: These are the rules the model (set of variables) must obey:

In [7]:
# SUBJECT TO:
C1= pp.LpConstraint(name='Gas Constraint',   # just the name
                    e= 1*Gas - 2*Oil, rhs=0, # linear combination of constraint and rhs
                    sense=pp.LpConstraintGE) # 'rule' >= 0 (LpConstraintGE)
C2= pp.LpConstraint(name='Oil Constraint',
                    e= 1*Oil, rhs=3000000,
                    sense=pp.LpConstraintGE) # 'rule' >= 3000000 (LpConstraintGE)
C3= pp.LpConstraint(name='Demand Constraint',
                    e= 1*Gas, rhs=6400000,
                    sense=pp.LpConstraintLE, )# 'rule' <= 6400000 (LpConstraintLE)

5. **Build MODEL**: Here you add (i) the objective function, and (ii) all the constraints:

In [8]:
model += obj_func
model += C1
model += C2
model += C3


6. **Solve the MODEL**: Notice we are not using the _default solver_, we are explicitly usig **COIN_CMD**:

In [9]:
solver_list = pp.listSolvers()
print(solver_list)

['GLPK_CMD', 'PYGLPK', 'CPLEX_CMD', 'CPLEX_PY', 'GUROBI', 'GUROBI_CMD', 'MOSEK', 'XPRESS', 'XPRESS', 'XPRESS_PY', 'PULP_CBC_CMD', 'COIN_CMD', 'COINMP_DLL', 'CHOCO_CMD', 'MIPCL_CMD', 'SCIP_CMD', 'FSCIP_CMD', 'SCIP_PY', 'HiGHS', 'HiGHS_CMD', 'COPT', 'COPT_DLL', 'COPT_CMD']


In [10]:
#solverToUse = pp.COIN_CMD(msg=False)
model.solve(0);

You can create a summary like this:

In [11]:
import pandas as pd

Results={"Model Status":pp.LpStatus[model.status]}
Results.update({"Optimal Solution":pp.value(model.objective)})
Results.update({v.name: v.varValue for v in model.variables()})
Results

{'Model Status': 'Optimal',
 'Optimal Solution': 16960000.0,
 'Gas': 6400000.0,
 'Oil': 3200000.0}

In [12]:
#or
pd.DataFrame.from_dict(Results,orient='index').T.set_index('Model Status').style.format('{:,}')

,Optimal Solution,Gas,Oil
Model Status,,,
Optimal,"16,960,000.0","6,400,000.0","3,200,000.0"


<div class="alert-success">

<strong>Exercise: The diet problem</strong>

In [13]:
%%html
<iframe src="https://docs.google.com/presentation/d/e/2PACX-1vTSq9X74urGAB_5n_MIJ9ZGIboKSvBdokVTBXVLh_qqZnmLRTJioOF431Rzys3Qi9UaFwWXjeq6Wmd5/embed?start=false&loop=false&delayms=3000" frameborder="0" width="960" height="569" allowfullscreen="true" mozallowfullscreen="true" webkitallowfullscreen="true"></iframe>

<div class="alert-success">

<strong>Exercise: The scheduling problem</strong>

In [14]:
%%html
<iframe src="https://docs.google.com/presentation/d/e/2PACX-1vQtBRpIr6Hx1_T0zJ3_DRqsE82YUjx7ZkeEKLdA64fbjtjkmc6Ibf6ebzp6CY69D482IGpG2h9GcsC5/embed?start=false&loop=false&delayms=3000" frameborder="0" width="960" height="569" allowfullscreen="true" mozallowfullscreen="true" webkitallowfullscreen="true"></iframe>

# Part 2: Multicriteria Decision-Making

In [15]:
%%html
<iframe src="https://docs.google.com/presentation/d/e/2PACX-1vR7GL_wF1eKRO0JgEUyIx5cxXUhTQ8ZM4F3AE1MLr7GYG33dwEobrLo6O2MaV2d7Cv47TaTgHghkhrV/embed?start=false&loop=false&delayms=3000" frameborder="0" width="960" height="569" allowfullscreen="true" mozallowfullscreen="true" webkitallowfullscreen="true"></iframe>

1. Prepare data file with the comparissons:

In [16]:
%%html

<iframe src="https://docs.google.com/spreadsheets/d/e/2PACX-1vSeUfh-DtfAAvEecirNS7Qs2qN4npmNfRiw9JvKmRpq88snVc8HJBlru2cyPy8lsQflSxlnx6U-IePw/pubhtml?widget=true&amp;headers=false" width="600" height="300" ></iframe>

2. Get the data (Excel)

In [17]:
# the link to the data

linkGoogle='https://docs.google.com/spreadsheets/d/e/2PACX-1vSeUfh-DtfAAvEecirNS7Qs2qN4npmNfRiw9JvKmRpq88snVc8HJBlru2cyPy8lsQflSxlnx6U-IePw/pub?output=xlsx'# the link to the data

3. Open each sheet:

In [18]:
# opening the comparissons

import pandas as pd

pairwise_age=pd.read_excel(linkGoogle,sheet_name='age', index_col=0)
pairwise_experience=pd.read_excel(linkGoogle,sheet_name='experience', index_col=0)
pairwise_education=pd.read_excel(linkGoogle,sheet_name='education', index_col=0)
pairwise_charisma=pd.read_excel(linkGoogle,sheet_name='charisma', index_col=0)
pairwise_criteria=pd.read_excel(linkGoogle,sheet_name='criteria', index_col=0)

You may want to check the structure:

In [19]:
pairwise_criteria

,experience,education,charisma,age
experience,1.000000,4.000000,3.000000,7
education,0.250000,1.000000,0.333333,3
charisma,0.333333,3.000000,1.000000,5
age,0.142857,0.333333,0.200000,1


4. Transform all matrices into pairwise comparissons:

In [20]:
import networkx as nx

G_age = nx.from_pandas_adjacency(pairwise_age,create_using=nx.MultiDiGraph())

# pairwise
G_age.edges(data=True)

OutMultiEdgeDataView([('Tom', 'Tom', {'weight': 1.0}), ('Tom', 'Dick', {'weight': 0.3333333333}), ('Tom', 'Harry', {'weight': 5.0}), ('Dick', 'Dick', {'weight': 1.0}), ('Dick', 'Tom', {'weight': 3.0}), ('Dick', 'Harry', {'weight': 9.0}), ('Harry', 'Harry', {'weight': 1.0}), ('Harry', 'Tom', {'weight': 0.2}), ('Harry', 'Dick', {'weight': 0.1111111111})])

In [21]:
# comparissons for age as dict
age_comparisons ={(e[0],e[1]):e[2]['weight'] for e in G_age.edges(data=True) if e[0]!= e[1]}
age_comparisons

{('Tom', 'Dick'): 0.3333333333,
 ('Tom', 'Harry'): 5.0,
 ('Dick', 'Tom'): 3.0,
 ('Dick', 'Harry'): 9.0,
 ('Harry', 'Tom'): 0.2,
 ('Harry', 'Dick'): 0.1111111111}

In [22]:
# the remaining comparissons:

G_exp = nx.from_pandas_adjacency(pairwise_experience,create_using=nx.MultiDiGraph())
experience_comparisons={(e[0],e[1]):e[2]['weight'] for e in G_exp.edges(data=True) if e[0]!= e[1]}

G_edu = nx.from_pandas_adjacency(pairwise_education,create_using=nx.MultiDiGraph())
education_comparisons={(e[0],e[1]):e[2]['weight'] for e in G_edu.edges(data=True) if e[0]!= e[1]}

G_cha = nx.from_pandas_adjacency(pairwise_charisma,create_using=nx.MultiDiGraph())
charisma_comparisons={(e[0],e[1]):e[2]['weight'] for e in G_cha.edges(data=True) if e[0]!= e[1]}

In [23]:
# take a look
[age_comparisons, experience_comparisons,education_comparisons,charisma_comparisons]

[{('Tom', 'Dick'): 0.3333333333,
  ('Tom', 'Harry'): 5.0,
  ('Dick', 'Tom'): 3.0,
  ('Dick', 'Harry'): 9.0,
  ('Harry', 'Tom'): 0.2,
  ('Harry', 'Dick'): 0.1111111111},
 {('Tom', 'Dick'): 0.25,
  ('Tom', 'Harry'): 4.0,
  ('Dick', 'Tom'): 4.0,
  ('Dick', 'Harry'): 9.0,
  ('Harry', 'Tom'): 0.25,
  ('Harry', 'Dick'): 0.1111111111},
 {('Tom', 'Dick'): 3.0,
  ('Tom', 'Harry'): 0.2,
  ('Dick', 'Tom'): 0.3333333333,
  ('Dick', 'Harry'): 0.1428571429,
  ('Harry', 'Tom'): 5.0,
  ('Harry', 'Dick'): 7.0},
 {('Tom', 'Dick'): 5.0,
  ('Tom', 'Harry'): 9.0,
  ('Dick', 'Tom'): 0.2,
  ('Dick', 'Harry'): 4.0,
  ('Harry', 'Tom'): 0.1111111111,
  ('Harry', 'Dick'): 0.25}]

In [24]:
# now the criteria

G_CRIT = nx.from_pandas_adjacency(pairwise_criteria,create_using=nx.MultiDiGraph())
criteria_comparisons ={(e[0],e[1]):e[2]['weight'] for e in G_CRIT.edges(data=True) if e[0]!= e[1]}
criteria_comparisons

{('experience', 'education'): 4.0,
 ('experience', 'charisma'): 3.0,
 ('experience', 'age'): 7.0,
 ('education', 'experience'): 0.25,
 ('education', 'charisma'): 0.3333333333,
 ('education', 'age'): 3.0,
 ('charisma', 'experience'): 0.3333333333,
 ('charisma', 'education'): 3.0,
 ('charisma', 'age'): 5.0,
 ('age', 'experience'): 0.1428571429,
 ('age', 'education'): 0.3333333333,
 ('age', 'charisma'): 0.2}

5. Apply the Algorithm

In [25]:
## install
# !pip install ahpy

In [26]:
# input each comparisson

import ahpy

experience = ahpy.Compare('experience', experience_comparisons, precision=3, random_index='saaty')
education = ahpy.Compare('education', education_comparisons, precision=3, random_index='saaty')
charisma = ahpy.Compare('charisma', charisma_comparisons, precision=3, random_index='saaty')
age = ahpy.Compare('age', age_comparisons, precision=3, random_index='saaty')
criteria = ahpy.Compare('criteria', criteria_comparisons, precision=3, random_index='saaty')

6. Create hierarchy:

In [27]:
criteria.add_children([experience, education, charisma, age])

7. See result:

In [28]:
print(criteria.target_weights)

{'Dick': np.float64(0.493), 'Tom': np.float64(0.358), 'Harry': np.float64(0.15)}


8. Assess consistency

In [29]:
## We should review comparissons if greater than 0.1!
[(val.name,val.consistency_ratio) for val in [experience, education, charisma, age, criteria]]

[('experience', np.float64(0.035)),
 ('education', np.float64(0.062)),
 ('charisma', np.float64(0.069)),
 ('age', np.float64(0.028)),
 ('criteria', np.float64(0.044))]

<div class="alert-success">

<strong>Exercise: Choosing a country for a Master Program</strong>

- Make a group of 4 people from this course.
- If you have the criteria: cost of living, language difficulty, possibilities to get a job in that country after studies are finished.
- If you have the alternatives: Brazil, Spain, USA, Germany.
- Create an AHP model and get the ranking.

You can follow this [example](https://en.wikipedia.org/wiki/Analytic_hierarchy_process_%E2%80%93_leader_example).
If you have a better idea, you can use it instead.

# Part 3: Benchmarking

Imagine you have this [information](https://www.sciencedirect.com/science/article/abs/pii/S0377221711007168):

In [30]:
airline=pd.read_csv("airlines_data.csv")
airline

FileNotFoundError: [Errno 2] No such file or directory: 'airlines_data.csv'

The first three variables (Aircraft,Fuel,Employee) represent **inputs** and the last two ones represent **outputs**. If that is so, there should be a way to compute some measure of efficiency: the ratio **output/input**.

Let's compute some ratios:

In [ ]:
# ratio passenger employee:
airline['rate_ClientsByEmployee']=(airline.Passenger/airline.Employee)
airline['rate_CargoByFleet']=(airline.Freight/airline.Aircraft)

Let me plot those ratios:

In [ ]:
import altair as alt

points = alt.Chart(airline).mark_point().encode(
    x='rate_ClientsByEmployee:Q',
    y='rate_CargoByFleet:Q'
)

text = points.mark_text(
    align='right',
    baseline='middle',
    dx=-7
).encode(
    text='name'
).interactive()

points + text

Which one is more efficient? As you see, one airline might not be good in both ratios:

In [ ]:
airline[['name','rate_ClientsByEmployee','rate_CargoByFleet']].sort_values(by='rate_ClientsByEmployee',ascending=False).head()

In [ ]:
airline[['name','rate_ClientsByEmployee','rate_CargoByFleet']].sort_values(by='rate_CargoByFleet',ascending=False).head()

Let me show you the **envelope**:

In [ ]:
Best_ClientsByEmployee=airline.rate_ClientsByEmployee.idxmax()
Best_CargoByFleet=airline.rate_CargoByFleet.idxmax()

frontier1=airline.loc[Best_ClientsByEmployee,['rate_ClientsByEmployee','rate_CargoByFleet']].to_list()
frontier2=airline.loc[Best_CargoByFleet,['rate_ClientsByEmployee','rate_CargoByFleet']].to_list()

#parallels
frontier1v=[frontier1[0],0]
frontier2h=[0,frontier2[1]]

#then
envelope=pd.DataFrame([frontier2h,frontier2,frontier1,frontier1v],columns=['x','y'])
envelope

Updating the plot:

In [ ]:
points + text + alt.Chart(envelope).mark_line(color='red').encode(
    x='x',
    y='y',
)

The presence of several units (DMUs), several inputs, and several outputs makes it difficult to judge who is doing better. This an optimization problem that may be carried out using **Pyfrontier**:

In [ ]:
## installation
# pip install Pyfrontier

Let me recover the names for each group of variables:

In [ ]:
airlineInput=airline.columns[1:4].to_list()
airlineOutput=airline.columns[4:6].to_list()

Let's apply the function:

In [ ]:
from Pyfrontier.frontier_model import EnvelopDEA

dea_air_vrs_in = EnvelopDEA("VRS", "in")
dea_air_vrs_in.fit(
    inputs=airline[airlineInput].to_numpy(),
    outputs=airline[airlineOutput].to_numpy()
)

Here is the result:

In [ ]:
airline['vrs_in']=[r.score for r in dea_air_vrs_in.result]
airline.set_index(airline.name,inplace=True)
airline['vrs_in']

At this stage, you may be tempted to do a regression:

In [ ]:
# !pip install py4etrics


In [ ]:
import numpy as np # linear algebra
from py4etrics import tobit
# import statsmodels.api as sm

airline['censored'] =np.where(airline['vrs_in']==1, 1, 0)
cens = airline['censored']
endog = airline.loc[:,'vrs_in']
exog = airline.loc[:,'Aircraft':'Freight']

tobit_res = tobit.Tobit(endog, exog, cens,right=1).fit()
tobit_res.summary()

<div class="alert-success">

<strong>Exercise: Efficiency in Public sector</strong>

- Make a group of 2 people from this course.
- Find a set of municipalities (homogenity required).
- Find a set of common input and out variables for them.
- Compute efficiency.